# Resources

- https://en.wikipedia.org/wiki/Customer_attrition
- https://public.tableau.com/profile/sandeep7769#!/vizhome/CustomerChurnAnalysis_4/TelecomCustomerChurnDataExploration
- https://developer.ibm.com/technologies/data-science/patterns/predict-customer-churn-using-watson-studio-and-jupyter-notebooks/#

# Import libraries

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic

sns.set_style("ticks")

from cleaning_pipeline import *

# Load data

In [ ]:
df = pd.read_csv("data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
df.head()

# Clean data

In [ ]:
cleaned_df = (
    df.pipe(start_pipeline)
    .pipe(drop_noisy_columns, cols=["customerID"])
    .pipe(replace_empty_strings_with_nan)
)

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.isna().sum() / len(df)

We can see that the percentage of missing values in the `TotalCharges` column is less than 1%, so we can drop all rows with missing values.

In [ ]:
cleaned_df = cleaned_df.pipe(drop_missing_values)

In [ ]:
cleaned_df

Now we can convert each column to the appropriate `dtype`.

In [ ]:
cleaned_df = cleaned_df.pipe(
    convert_column_dtypes, {"SeniorCitizen": "str", "TotalCharges": np.float}
)

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.dtypes

Our final cleaning and processing pipeline would be:

In [ ]:
cleaned_df = (
    df.pipe(start_pipeline)
    .pipe(drop_noisy_columns, cols=["customerID"])
    .pipe(replace_empty_strings_with_nan)
    .pipe(drop_missing_values)
    .pipe(map_column_values, col="SeniorCitizen", mapping_dict={0: "No", 1: "Yes"})
    .pipe(convert_column_dtypes, dtypes_mapping={"TotalCharges": np.float},)
)

In [ ]:
cleaned_df.head()

# Exploratory Data Analysis

In [ ]:
demographic_cols = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
]

In [ ]:
account_cols = [
    "tenure",
    "Contract",
    "PaymentMethod",
    "PaperlessBilling",
    "MonthlyCharges",
    "TotalCharges",
]

In [ ]:
services_cols = [
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
]

## Target variable

In [ ]:
temp_df = cleaned_df.groupby(by="Churn", as_index=False).agg(
    count=pd.NamedAgg(column="Churn", aggfunc="count")
)

temp_df["count"] = temp_df["count"].map(lambda x: x / len(cleaned_df))

In [ ]:
fig = px.bar(data_frame=temp_df, x="Churn", y="count", color="Churn")

fig.show()

## Demographic attributes

### `Gender`

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="gender", color="gender")

fig.show()

### `SeniorCitizen`

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="SeniorCitizen", color="SeniorCitizen")

fig.show()

### `Partner`

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="Partner", color="Partner")

fig.show()

### `Dependents`

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="Dependents", color="Dependents")

fig.show()

## Account attributes

### `tenure`

In [ ]:
cleaned_df.tenure.head()

In [ ]:
df.tenure.describe()

In [ ]:
df.tenure.skew()

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="tenure", marginal="box")

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df, x="tenure", color="Churn", barmode="group", marginal="box"
)

fig.show()

We can conclude from this plot some important insights:
- New customers with about 3 months of tenure are the most likely to churn.
- The chart also shows that the higher customer tenure is, the less likely is he to churn.
- There are some extreme points (outliers) for the *churning* customers, where they have a very high tenure (70 months, almost 6 years), yet they churn from the company. These might be *outliers*, or this could be related to other factors.

### `Contract`

In [ ]:
cleaned_df.Contract.head()

In [ ]:
temp_df = (
    cleaned_df.groupby(by="Contract", as_index=False)
    .agg(count=pd.NamedAgg(column="Contract", aggfunc="count"))
    .sort_values(by="count", ascending=False)
)

In [ ]:
temp_df

In [ ]:
fig = px.bar(data_frame=temp_df, x="Contract", y="count", color="Contract")

fig.show()

The majority of customers prefer *monthly* contracts.

Let's see the relation between contract type and whether customer churn the company or not:

In [ ]:
temp_df = cleaned_df.groupby(by=["Contract", "Churn"], as_index=False).size().sort_values(
    by="size", ascending=False
)

In [ ]:
fig = px.bar(data_frame=temp_df, x='Contract', y='size', color='Churn', barmode='group')

fig.show()

This chart suggests that customers who use short term contracts are far more likely to leave the service.

### `PaymentMethod`